## Working with S2 in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/02_22.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

For working with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/), [vgridpandas document](https://vgridpandas.gishub.vn)

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/) if needed.

In [ ]:
# %pip install vgrid --upgrade

### latlon2s2

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2s2

lat = 10.775276
lon = 106.706797
res = 12
s2_token = latlon2s2(lat, lon, res)
s2_token

### S2 to Shapely Polygons

In [ ]:
from vgrid.conversion.dggs2geo.s22geo import s22geo

s2_geo = s22geo(s2_token)
s2_geo

### S2 to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.s22geo import s22geojson

s2_geojson = s22geojson(s2_token)
s2_geojson

### Vector to S2

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.vector2dggs.vector2s2 import vector2s2

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
s2_geojson = vector2s2(file_path, resolution=17, compact=False, predicate = "intersects")
# Change compact to True to see the compact version of the rHEALPix cells

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

s2_layer = folium.GeoJson(
s2_geojson,
style_function=lambda x: {
    "fillColor": "blue",
    "color": "black",
    "fillOpacity": 0.3,
    "weight": 1,
},
tooltip=folium.GeoJsonTooltip(
    fields=["s2", "resolution", "cell_area"],
    aliases=["S2 Token", "Resolution", "Area (m²)"],
    style="""
        background-color: white;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
),
).add_to(m)

m.fit_bounds(s2_layer.get_bounds())

# Display the map
m

### S2 Compact

In [ ]:
from vgrid.conversion.dggscompact import s2compact

s2_compacted = s2compact(
    s2_geojson, s2_token="s2"
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

s2_compacted_layer = folium.GeoJson(
    s2_compacted,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["s2", "resolution", "cell_area"],
        aliases=["S2 Token", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(s2_compacted_layer.get_bounds())

# Display the map
m

### S2 Expand

In [ ]:
from vgrid.conversion.dggscompact import s2expand

s2_expanded = s2expand(
    s2_geojson, resolution=18
)

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles="CartoDB positron")

s2_expanded_layer = folium.GeoJson(
    s2_expanded,
    style_function=lambda x: {
        "fillColor": "blue",
        "color": "black",
        "fillOpacity": 0.3,
        "weight": 1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["s2", "resolution", "cell_area"],
        aliases=["S2 Token", "Resolution", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(s2_expanded_layer.get_bounds())

# Display the map
m

### S2 Binning

In [ ]:
from vgrid.binning.s2bin import s2bin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
s2_bin = s2bin(file_path, resolution=15, stats="count")
s2_bin 

### S2 Grid Generator

In [ ]:
from vgrid.generator.s2grid import s2grid

s2_grid = s2grid(resolution=3,output_format="gpd")
# s2_grid = s2grid(resolution=16,bbox=[106.699007, 10.762811, 106.717674, 10.778649],output_format="gpd")  
s2_grid.plot(edgecolor='white')

### S2 Statistics

In [ ]:
from vgrid.stats.s2stats import s2stats
s2stats()